# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle



# Modulos da solucao
import modules.cronometro as cron
import modules.utils as utils
import modules.trata_pdf as tpdf


In [2]:
#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"



#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

### Funcoes para Analise do pipeline

In [3]:
# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "NO_PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [4]:
# SOMENTE PARA TESTE DE FUNCAO
fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = utils.busca_proximo_batch(conf_export_plan_path)
batch_name

'Batch_23'

In [ ]:
documentos_extracao_path

In [5]:
batch_name = 'Batch_21'

In [6]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            nome_arquivo = file
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = tpdf.analisa_nro_pages(file_path)
                one_page = doc_one_page
                num_page = nro_pgs
            else:
                one_page = False
                num_page = 0    
            # if doc_one_page:
            #     one_page = doc_one_page
            # else:
            #     one_page = False    
            #             rotulo = 'pdf_mul_paginas'
            level = 3
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": num_page,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "document_unique_id": utils.generate_unique_id(),
                "parent_document_unique_id": fake_parent_document_unique_id,
                "file_hash": utils.generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [7]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status)

In [8]:
df_root_pipe

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,pages,palavra_chave,document_tag,action_item,level,document_unique_id,parent_document_unique_id,file_hash,file_path,informations
0,1,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf,11756286,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,c1684259-4662-4b48-8fa9-e8f259ced1e4,f976c128-1f41-4551-bffd-fac687c1c8b2,b3acfffea4847108d0064ecbd62a73359961f88741a037...,pipeline_extracao_documentos/2_documentos_para...,
1,2,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,Nota Fiscal Eletrônica Quallit 24072023.pdf,11766341,True,1,nota,prov_nota_fiscal,NO_PROCESS,3,dd30e887-a8c4-4ac0-86e4-8353610d5621,f976c128-1f41-4551-bffd-fac687c1c8b2,ae37292a66dff093838f3cde0da8bea332ec81241b1d5f...,pipeline_extracao_documentos/2_documentos_para...,
2,3,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,NFE 20237.pdf,11624359,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,0b740f38-6eb5-4f13-a8de-20234fd80f12,f976c128-1f41-4551-bffd-fac687c1c8b2,1976ffe84e27b6bb1a5840018c4a47f6bac483b10f686a...,pipeline_extracao_documentos/2_documentos_para...,
3,4,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,Ampla.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,bb21cd9b-cbfd-4519-9582-f3cbfa4352bf,f976c128-1f41-4551-bffd-fac687c1c8b2,ad2522b53349fffd748376bca7fe4b90fd59359971d992...,pipeline_extracao_documentos/2_documentos_para...,
4,5,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,Supernova.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,f88f4716-f7cb-47a7-83e3-34179872ae63,f976c128-1f41-4551-bffd-fac687c1c8b2,c26954dad71d508d4b5315f69dc4c6291a1c5bf70548e7...,pipeline_extracao_documentos/2_documentos_para...,
5,6,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,Blue Lord.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,6ca52928-954f-4e67-ab12-5bd37fcebc7b,f976c128-1f41-4551-bffd-fac687c1c8b2,63b6603be7b1df359e67b96e9e128336b46fa3fbcfdadc...,pipeline_extracao_documentos/2_documentos_para...,
6,7,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,NF 2023158.pdf,11285853,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,9199a613-0d5c-43b5-bbf7-24ef4125c5d3,f976c128-1f41-4551-bffd-fac687c1c8b2,abcfcce04cabd261b30730f04cbb98e030f2a765bb31aa...,pipeline_extracao_documentos/2_documentos_para...,
7,8,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,NF 2023157.pdf,11285853,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,126bb763-3b98-4678-b3cf-3d05bff3bcb8,f976c128-1f41-4551-bffd-fac687c1c8b2,c8c5ae8ccca5e9befb6445e21701dc99266746e877c630...,pipeline_extracao_documentos/2_documentos_para...,
8,9,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,nota_07_2023.pdf,11778425,True,1,nota,prov_nota_fiscal,NO_PROCESS,3,7ec7825d-1aac-435e-a9a9-934acdd36e08,f976c128-1f41-4551-bffd-fac687c1c8b2,33bc08587f5c227ab3e035011ed9b20340f99599e65ff7...,pipeline_extracao_documentos/2_documentos_para...,
9,10,23/09/2023 18:02:54,Batch_21,analise,scan_analise,root_analise,Analise,nf 59.pdf,11359989,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,c04530d9-0e92-4a4b-b2d4-f6d5b1a0bbb9,f976c128-1f41-4551-bffd-fac687c1c8b2,59274230e7fb7897397bb426ac31d953e2a14def1fd25e...,pipeline_extracao_documentos/2_documentos_para...,


### Manipulando od DF

In [ ]:
# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [ ]:
# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]

In [ ]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_analise_pipe_path = "Em_HOLD/df_mapeamento_e_analise2.xlsx"


#Le a planilha e cria df_documento_recebido
df_analise_pipe = pd.read_excel(df_analise_pipe_path)

# Ajusta o indice
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
# 2B. XXX Efetuo a analise do pipeline de documentos e inicio da extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'Reavaliar_PDF_Pesquisavel' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'mapear'

imagens_list = analise_extracao_pipeline(subset_df_analise_pipe, df_analise_pipe, fase, atividade, status)


if imagens_list:
    remove_images(imagens_list)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

### exportando o map_analise_path

In [ ]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)